In [5]:
!wget http://www.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
!unzip shakespeare.zip

URL transformed to HTTPS due to an HSTS policy
--2023-10-11 22:39:18--  https://www.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www.lawrence.edu (www.lawrence.edu)... 143.44.124.14
Connecting to www.lawrence.edu (www.lawrence.edu)|143.44.124.14|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www7.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip [following]
--2023-10-11 22:39:18--  https://www7.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www7.lawrence.edu (www7.lawrence.edu)... 143.44.124.14
Connecting to www7.lawrence.edu (www7.lawrence.edu)|143.44.124.14|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www2.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip [following]
--2023-10-11 22:39:18--  https://www2.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www2.lawrence.edu (www2.lawrence.edu)... 143.44.124.14
Connecting to www2.lawrence.edu (www2.lawrence.edu)|

In [8]:
def getWord(file):
    char = file.read(1)
    if not char:
        return ''
    char = char.lower()

    while char < 'a' or char > 'z':
        char = file.read(1)        
        if not char: 
            return ''
        char = char.lower()

    str = ''
    while char >= 'a' and char <= 'z':
        str = str + char
        char = file.read(1)        
        if not char: 
            return str
        char = char.lower()
    
    if char == '’':
        str = ''
        char = file.read(1)
        if not char: 
            return str
        char = char.lower()
        while char >= 'a' and char <= 'z':
            char = file.read(1)        
            if not char: 
                return str
            char = char.lower()
        return getWord(file)
    return str

In [9]:
import numpy as np

def encode(ch):
    result = np.zeros(27)
    if ch == ' ':
        position = 26
    else:
        position = ord(ch) - ord('a')
    result[position] = 1
    return result

In [17]:
encode(' ')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [26]:
def makeSequence(str):
    sequence = np.zeros((10,27))
    for n in range(0,len(str)):
        sequence[n] = encode(str[n])
    for n in range(len(str), 10):
        sequence[n] = encode(' ')
    return sequence


In [28]:
file = open('shakespeare.txt', 'r')

inputs = np.zeros((100000, 10, 27))
targets = np.zeros((100000, 27))
counter = 0
while(True):
    word = getWord(file)
    if len(word) > 4 and len(word) < 9:    
        for n in range(3, len(word)):
            inputs[counter] = makeSequence(word[:n])
            targets[counter] = encode(word[n])
            counter += 1 
            if counter == 100000:
                break
    if counter == 100000:
        break

file.close()

In [36]:
targets[-1]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [44]:
train_data = inputs[:64000]
train_targets = targets[:64000]
validation_data = inputs[64000:80000]
validation_targets = targets[64000:80000]
test_data = inputs[80000:]
test_targets = targets[80000:]


In [30]:
train_targets[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [45]:
from tensorflow.keras import layers
from tensorflow.keras import Sequential

model = Sequential([layers.LSTM(26,input_shape=(10,27),return_sequences=False),layers.Dense(27, activation="softmax")])
#x =layers.LSTM(16,input_shape=(10,27),return_sequences=False)
#outputs = layers.Dense(27, activation="softmax")(x)
#model = keras.Model(x, outputs)
model.compile(optimizer="rmsprop",
            loss="categorical_crossentropy",
            metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 26)                5616      
                                                                 
 dense_4 (Dense)             (None, 27)                729       
                                                                 
Total params: 6345 (24.79 KB)
Trainable params: 6345 (24.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
validation_data[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],


In [46]:
from tensorflow import keras
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.mdl",
                                    save_best_only=True)
]
model.fit(train_data, train_targets, validation_data=[validation_data,validation_targets], epochs=10, callbacks=callbacks)


Epoch 1/10
1998/2000 [============================>.] - ETA: 0s - loss: 2.6199 - accuracy: 0.2159INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


2000/2000 [==============================] - 16s 8ms/step - loss: 2.6195 - accuracy: 0.2160 - val_loss: 2.3753 - val_accuracy: 0.2753
Epoch 2/10
1995/2000 [============================>.] - ETA: 0s - loss: 2.2250 - accuracy: 0.3359INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


2000/2000 [==============================] - 14s 7ms/step - loss: 2.2246 - accuracy: 0.3360 - val_loss: 2.1998 - val_accuracy: 0.3630
Epoch 3/10
1999/2000 [============================>.] - ETA: 0s - loss: 2.0100 - accuracy: 0.4136INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


...

2000/2000 [==============================] - 15s 7ms/step - loss: 1.6218 - accuracy: 0.5336 - val_loss: 1.9299 - val_accuracy: 0.4316
Epoch 8/10
1996/2000 [============================>.] - ETA: 0s - loss: 1.5699 - accuracy: 0.5543INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


2000/2000 [==============================] - 15s 7ms/step - loss: 1.5694 - accuracy: 0.5544 - val_loss: 1.9106 - val_accuracy: 0.4340
Epoch 9/10
2000/2000 [==============================] - 13s 6ms/step - loss: 1.5259 - accuracy: 0.5679 - val_loss: 1.9123 - val_accuracy: 0.4435
Epoch 10/10
1998/2000 [============================>.] - ETA: 0s - loss: 1.4904 - accuracy: 0.5803INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


INFO:tensorflow:Assets written to: embeddings_bidir_gru.mdl/assets


2000/2000 [==============================] - 16s 8ms/step - loss: 1.4901 - accuracy: 0.5803 - val_loss: 1.8582 - val_accuracy: 0.4589


In [47]:
model = keras.models.load_model("embeddings_bidir_gru.mdl")
print(f"Test acc: {model.evaluate(test_data,test_targets)[1]:.3f}")

625/625 [==============================] - 1s 1ms/step - loss: 1.9744 - accuracy: 0.4389
Test acc: 0.439
